In [1]:
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import numpy as np

from datetime import datetime
from datetime import timedelta

import pandas as pd

In [2]:
from lib.dataeng import Etl

In [3]:
etl = Etl()

etl.repo

'/home/benjamin/Downloads/'

In [4]:
df_weight_raw, df_food_raw, df_exos_raw, df_all = etl.extract_store_all()


Le fichier poids le plus récent est : data_BEN_1709441711.zip, parmi :
data_BEN_1709364593.zip
data_BEN_1709185019.zip
data_BEN_1709012531.zip
data_BEN_1709441711.zip
data_BEN_1709272584.zip
data_BEN_1709103824.zip

Le fichier food le plus récent est : File-Export-2020-08-03-to-2024-03-03.zip, parmi
File-Export-2020-08-03-to-2024-02-29.zip
File-Export-2020-08-03-to-2024-02-27.zip
File-Export-2020-08-03-to-2024-02-28.zip
File-Export-2020-08-03-to-2024-03-02.zip
File-Export-2020-08-03-to-2024-03-03.zip
File-Export-2020-08-03-to-2024-03-01.zip

Le fichier exercices le plus récent est : polar-user-data-export_bd8a06c1-7cac-47fc-a1fe-71b374e61732.zip, parmi :
polar-user-data-export_bd8a06c1-7cac-47fc-a1fe-71b374e61732.zip
polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip


100%|██████████| 1924/1924 [01:33<00:00, 20.62it/s]


In [5]:
df_weight = df_weight_raw.groupby('Date').mean()

In [6]:
df_std = df_weight_raw.groupby('Date').std()

for colname in df_weight.columns:
    std_colname = 'std_' + colname
    df_weight[std_colname] = df_std[colname]

In [7]:
df_weight.dropna(inplace=True)

df_weight

,Masse_Totale,Masse_Grasse,Masse_Osseuse,Masse_Musculaire,Masse_Hydrique,MG%,BMR,std_Masse_Totale,std_Masse_Grasse,std_Masse_Osseuse,std_Masse_Musculaire,std_Masse_Hydrique,std_MG%,std_BMR
Date,,,,,,,,,,,,,,
2023-03-04,70.656667,11.420000,2.961667,56.258333,39.496667,0.161627,1649.512000,0.036148,0.058992,0.004082,0.056716,0.066533,0.000812,1.264415
2023-03-05,71.880000,11.848571,3.000000,57.014286,40.180000,0.164837,1666.678857,0.092376,0.106369,0.000000,0.037353,0.053229,0.001281,0.821828
2023-03-06,71.030000,10.924000,3.010000,57.082000,40.476000,0.153794,1668.289600,0.023452,0.030496,0.000000,0.031937,0.040988,0.000427,0.787734
2023-03-07,72.185000,11.126667,3.060000,57.986667,41.366667,0.154141,1688.860000,0.018708,0.050067,0.000000,0.052026,0.072019,0.000706,1.255156
2023-03-08,72.283333,10.178333,3.131667,58.956667,42.821667,0.140811,1711.468000,0.042269,0.052313,0.004082,0.038297,0.064317,0.000674,0.893206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-28,72.064667,12.396667,2.982667,56.672000,39.747333,0.172019,1658.828800,0.151510,0.132701,0.004577,0.077294,0.078510,0.001547,1.760859
2024-02-29,71.842000,12.396000,2.970667,56.460000,39.540000,0.172544,1654.033600,0.135657,0.083049,0.002582,0.061296,0.051824,0.000855,1.370488
2024-03-01,71.987333,11.926000,3.004000,57.042667,40.192667,0.165667,1667.324800,0.135249,0.092952,0.005071,0.076669,0.077318,0.001056,1.756437


In [8]:
PAST = 40 # on regarde les N derniers jours
PAST_REG = 30 # horizon de temps passé pour la régression
FUTUR = 30 # vers les M prochains jours
MOY = 1

df_weight2 = df_weight.rolling(window=MOY).mean()[-PAST:]

df_food2 = df_food_raw.groupby('Date').sum(numeric_only=True).rolling(window=MOY).mean()[-PAST:]

df_exos2 = df_exos_raw.groupby('exo_date').sum(numeric_only=True)
BMR = df_weight['BMR'][-PAST:].mean()
df_exos2['exo_cals_nets'] = df_exos2['exo_cals_bruts'] - df_exos2['exo_duree']/3600*(BMR/24)
df_exos2 = df_exos2.rolling(window=MOY).mean()[-PAST:]

In [9]:
fig, ax = plt.subplots(figsize=(24,12), nrows=3, ncols=2)

list_to_display = ['Masse_Totale', 'Masse_Grasse', 'MG%', 'Masse_Musculaire']

for i, name in enumerate(list_to_display):
    r = i//2
    c = i%2
    name_std = 'std_' + name

    # historique avec standard deviation
    lbl = name + f' moyenne sur {MOY} jour(s)'
    ax[r,c].plot(df_weight2[name], marker = 'x', color='blue', label=lbl)
    ax[r,c].fill_between(
            df_weight2.index,
            df_weight2[name] - 1.96 * df_weight2[name_std],
            df_weight2[name] + 1.96 * df_weight2[name_std],
            alpha=0.8,
            label=r"Historique 95% confidence interval",
        )
    ax[r,c].grid(True)

    # tendance
    full_std = df_weight2[name_std].mean()
    # projection LR
    lrr = LinearRegression()
    X_past = np.array( [ d.toordinal() for d in df_weight2.index[-PAST_REG:] ] ).reshape(-1,1)
    y = df_weight2[name][-PAST_REG:]
    lrr.fit(X_past, y)
    if name=='MG%':
        lbl += f' tendance LR = {lrr.coef_[0]*30*100:.2f}%/mo'
    else:
        lbl += f' tendance LR = {lrr.coef_[0]*30:.3f}/mo'
    # calcul tendance
    X_trend = [ df_weight2.index[-1] + timedelta(days=i) for i in range(-PAST_REG, FUTUR+1 )]
    X_trend = np.array( [ d.toordinal() for d in X_trend ] ).reshape(-1,1)
    y_pred = lrr.predict(X_trend)

    X_trend_days = [ datetime.fromordinal(X_trend[i][0]) for i in range(len(X_trend)) ]
    df_pred = pd.DataFrame( data={"tendance" : y_pred}, index=X_trend_days )
    ax[r,c].plot(df_pred, marker='o', color='green')

    ax[r,c].fill_between(
            df_pred.index,
            df_pred['tendance'] - 1.96 * full_std,
            df_pred['tendance'] + 1.96 * full_std,
            alpha=0.8,
            label=r"Tendance 95% confidence interval",
    )

    ax[r,c].legend()
    ax[r,c].set_title(lbl)
    
fig.suptitle(f'Tendances')

# -- moyenne calories --

xmin = df_food2.index[0]
xmax = df_food2.index[-1]

# ax[2,0].plot(df_food2['Calories'])
ax[2,0].bar(x=df_food2['Calories'].index, height=df_food2['Calories'].to_numpy())
# df_food2['Calories'].plot.bar(ax=ax[2,0])
cal_moy = df_food2['Calories'].mean()
# ax[2,0].hlines(cal_moy, xmin=[xmin], xmax=[xmax], color='green')
ax[2,0].plot([xmin, xmax], [cal_moy, cal_moy], color='green')
ax[2,0].set_title(f'calories food - moyenne {PAST} derniers jours = {cal_moy:.0f}')
ax[2,0].grid(True)

# -- moyenne exercices ----

# ax[2,1].plot(df_exos2['exo_cals_nets'])
ax[2,1].bar(x=df_exos2['exo_cals_nets'].index, height=df_exos2['exo_cals_nets'].to_numpy())
exo_nets = df_exos2['exo_cals_nets'].mean()
# ax[2,1].hlines(exo_nets, xmin=xmin, xmax=xmax, color='green')
ax[2,1].plot([xmin, xmax], [exo_nets, exo_nets], color='green')
ax[2,1].set_title(f'calories net exercices - moyenne {PAST} derniers jours = {exo_nets:.0f}')
ax[2,1].grid(True)

plt.show()